**About** : This notebook is used to prepare the data.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

### Imports

In [ ]:
import os
import sys
import glob
import json
import h5py
import librosa
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from copy import deepcopy
from joblib import delayed
from sklearn.metrics import *

warnings.simplefilter(action="ignore", category=UserWarning)
pd.set_option('display.width', 500)
pd.set_option('max_colwidth', 100)

In [ ]:
from params import *
from data.preparation import *
from util.plots import display_audio
from data.processing import create_target_path, ProgressParallel, get_load_librosa_save_h5py

## Competiton data

### Data

In [ ]:
# df = prepare_data()
df = prepare_data_2()

In [ ]:
# prepare_folds()

In [ ]:
df.head()

### Convert to h5py

In [ ]:
SAVE_FOLDER = DATA_PATH + "train_features/"

In [ ]:
!rm -rf $SAVE_FOLDER

In [ ]:
audios = glob.glob(DATA_PATH + "train_audio/*/*")
print(f"-> Found {len(audios)} files")

In [ ]:
targets = [create_target_path(SAVE_FOLDER, f) for f in audios]

os.makedirs(SAVE_FOLDER, exist_ok=True)
for f in set([os.path.dirname(f) for f in targets]):
    os.makedirs(f, exist_ok=True)

print(f"-> Created {len(os.listdir(SAVE_FOLDER))} folders")

In [ ]:
fct = get_load_librosa_save_h5py(do_normalize=False, sr=32000)
# fct(audios[0], targets[0])

In [ ]:
_ = ProgressParallel(n_jobs=16, total=len(audios))(
    delayed(fct)(load_path, save_path)
    for load_path, save_path in zip(audios, targets)
)

In [ ]:
saved_targets = glob.glob(SAVE_FOLDER + "*/*.hdf5")
print(f"Saved {len(saved_targets)} files")

In [ ]:
df.head()

## Xenocanto extra data

### Data

In [ ]:
df_xc = prepare_xenocanto_data()

In [ ]:
df_xc = add_xeno_low_freq(df, upsample_to=0, low_freq=500).reset_index(drop=True)

### Convert to h5py

In [ ]:
audios = glob.glob(DATA_PATH + "xenocanto/audio/*/*")
audios = [a for a in audios if a.split("/", 4)[-1] in df_xc.filename.values]
print(f"-> Found {len(audios)} files")

In [ ]:
SAVE_FOLDER = DATA_PATH + "xenocanto/features/"

In [ ]:
!rm -rf $SAVE_FOLDER

In [ ]:
targets = [create_target_path(SAVE_FOLDER, f) for f in audios]

os.makedirs(SAVE_FOLDER, exist_ok=True)
for f in set([os.path.dirname(f) for f in targets]):
    os.makedirs(f, exist_ok=True)

print(f"-> Created {len(os.listdir(SAVE_FOLDER))} folders")

In [ ]:
fct = get_load_librosa_save_h5py(do_normalize=False, sr=32000)
fct(audios[0], targets[0])

In [ ]:
_ = ProgressParallel(n_jobs=16, total=len(audios))(
    delayed(fct)(load_path, save_path)
    for load_path, save_path in zip(audios, targets)
)

In [ ]:
saved_targets = glob.glob(SAVE_FOLDER + "*/*.hdf5")
print(f"Saved {len(saved_targets)} files")

### Nocall

In [ ]:
paths = [
    DATA_PATH + "background_noise/birdclef2021_nocall/",
    DATA_PATH + "background_noise/birdclef2020_nocall/",
    DATA_PATH + "background_noise/freefield/",
    DATA_PATH + "background_noise/warblrb/",
    DATA_PATH + "background_noise/birdvox/",
    DATA_PATH + "background_noise/rainforest/",
    DATA_PATH + "background_noise/environment/",
    DATA_PATH + "nocall_dieter/aicrowd2020_noise_30sec/noise_30sec/",
    DATA_PATH + "nocall_dieter/ff1010bird_nocall/nocall/",
    DATA_PATH + "nocall_dieter/train_soundscapes/nocall/",
]

In [ ]:
max_samples = 200

dfs = []
for path in paths:
    audios = glob.glob(path + "*")
    df = pd.DataFrame({"path": audios})

    if len(df) > max_samples:
        df = df.sample(max_samples)

    df["duration"] = df["path"].apply(lambda x: librosa.get_duration(path=x))

    src = path.split("/", 3)[-1].split('/')[0]
    df["source"] = src

    df = df[df['duration'] >= 5]
    df = df[df['duration'] <= 60]

    dfs.append(df)

    print(f"-> Found {len(audios)} files in {src}")

In [ ]:
df = pd.concat(dfs, ignore_index=True)

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
sns.countplot(y=df['source'].values)
plt.subplot(1, 2, 2)
sns.histplot(x=df['duration'].values)
plt.show()

In [ ]:
SAVE_FOLDER = DATA_PATH + "nocall_features/"
audios = df["path"].values

In [ ]:
targets = [create_target_path(SAVE_FOLDER, f) for f in audios]

os.makedirs(SAVE_FOLDER, exist_ok=True)
for f in set([os.path.dirname(f) for f in targets]):
    os.makedirs(f, exist_ok=True)

print(f"-> Created {len(os.listdir(SAVE_FOLDER))} folders")

In [ ]:
fct = get_load_librosa_save_h5py(do_normalize=False, sr=32000)
fct(audios[0], targets[0])

In [ ]:
_ = ProgressParallel(n_jobs=16, total=len(audios))(
    delayed(fct)(load_path, save_path)
    for load_path, save_path in zip(audios, targets)
)

In [ ]:
saved_targets = glob.glob(SAVE_FOLDER + "*/*.hdf5")
print(f"Saved {len(saved_targets)} files")

## Unlabeled data

### Data

### Convert to h5py

In [ ]:
audios = glob.glob(DATA_PATH + "unlabeled_soundscapes/*")
print(f"-> Found {len(audios)} files")

In [ ]:
SAVE_FOLDER = DATA_PATH + "unlabeled_features/"

In [ ]:
!rm -rf $SAVE_FOLDER

In [ ]:
targets = [create_target_path(SAVE_FOLDER, f) for f in audios]

os.makedirs(SAVE_FOLDER, exist_ok=True)
for f in set([os.path.dirname(f) for f in targets]):
    os.makedirs(f, exist_ok=True)

print(f"-> Created {len(os.listdir(SAVE_FOLDER))} folders")

In [ ]:
fct = get_load_librosa_save_h5py(do_normalize=False, sr=32000)
fct(audios[0], targets[0])

In [ ]:
_ = ProgressParallel(n_jobs=16, total=len(audios))(
    delayed(fct)(load_path, save_path)
    for load_path, save_path in zip(audios, targets)
)

In [ ]:
saved_targets = glob.glob(SAVE_FOLDER + "*/*.hdf5")
print(f"Saved {len(saved_targets)} files")

### Nocall

In [ ]:
paths = [
    DATA_PATH + "background_noise/birdclef2021_nocall/",
    DATA_PATH + "background_noise/birdclef2020_nocall/",
    DATA_PATH + "background_noise/freefield/",
    DATA_PATH + "background_noise/warblrb/",
    DATA_PATH + "background_noise/birdvox/",
    DATA_PATH + "background_noise/rainforest/",
    DATA_PATH + "background_noise/environment/",
    DATA_PATH + "nocall_dieter/aicrowd2020_noise_30sec/noise_30sec/",
    DATA_PATH + "nocall_dieter/ff1010bird_nocall/nocall/",
    DATA_PATH + "nocall_dieter/train_soundscapes/nocall/",
]

In [ ]:
max_samples = 200

dfs = []
for path in paths:
    audios = glob.glob(path + "*")
    df = pd.DataFrame({"path": audios})

    if len(df) > max_samples:
        df = df.sample(max_samples)

    df["duration"] = df["path"].apply(lambda x: librosa.get_duration(path=x))

    src = path.split("/", 3)[-1].split('/')[0]
    df["source"] = src

    df = df[df['duration'] >= 5]
    df = df[df['duration'] <= 60]

    dfs.append(df)

    print(f"-> Found {len(audios)} files in {src}")

In [ ]:
df = pd.concat(dfs, ignore_index=True)

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
sns.countplot(y=df['source'].values)
plt.subplot(1, 2, 2)
sns.histplot(x=df['duration'].values)
plt.show()

In [ ]:
SAVE_FOLDER = DATA_PATH + "nocall_features/"
audios = df["path"].values

In [ ]:
targets = [create_target_path(SAVE_FOLDER, f) for f in audios]

os.makedirs(SAVE_FOLDER, exist_ok=True)
for f in set([os.path.dirname(f) for f in targets]):
    os.makedirs(f, exist_ok=True)

print(f"-> Created {len(os.listdir(SAVE_FOLDER))} folders")

In [ ]:
fct = get_load_librosa_save_h5py(do_normalize=False, sr=32000)
fct(audios[0], targets[0])

In [ ]:
_ = ProgressParallel(n_jobs=16, total=len(audios))(
    delayed(fct)(load_path, save_path)
    for load_path, save_path in zip(audios, targets)
)

In [ ]:
saved_targets = glob.glob(SAVE_FOLDER + "*/*.hdf5")
print(f"Saved {len(saved_targets)} files")

### Duplicates

In [ ]:
df_tot = pd.concat([df, df_xc])
dups = df_tot[df_tot.duplicated(subset='id', keep=False)].sort_values('id')

In [ ]:
# dups[dups['fold'] == -1].id.values.tolist()

In [ ]:
dups.head(10)

In [ ]:
idx = 0

In [ ]:
display_audio(dups["path"].values[idx])

In [ ]:
display_audio(dups["path"].values[idx + 1])

Done !